# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome()

#Passo1: Pegar a cotação do dólar
#Entrar no google
navegador.get("https://www.google.com/search?q=cota%C3%A7%C3%A3o+d%C3%B3lar&oq=cota&aqs=chrome.1.69i59l2j69i57j69i61.2797j0j4&sourceid=chrome&ie=UTF-8")

cotacao_dolar = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(cotacao_dolar)
                                                                                                                                               
#Passo2: Pegar a cotação do euro
navegador.get("https://www.google.com/search?q=cota%C3%A7%C3%A3o+euro&oq=cota&aqs=chrome.0.69i59l2j69i57j69i61.1893j0j9&sourceid=chrome&ie=UTF-8")

cotacao_euro = navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

print(cotacao_euro)
#Passo3: Pegar a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')

cotacao_ouro = cotacao_ouro.replace(",", ".")

print(cotacao_ouro)

navegador.quit()

5.1087
5.5465
312.26


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [2]:
#Passo4: Importar a base de dados e Atualizar a base
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)



,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [3]:
#Passo5: Recalcular os preços

#atualizar a cotação
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

#preco de compra = cotação * preco original
tabela["Preço de Compra"] = tabela["Cotação"] * tabela["Preço Original"]

#preco de venda = preco de compra * margem
tabela["Preço"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda,Preço
0,Câmera Canon,999.99,Dólar,5.1087,5108.648913,1.40,6999.930,7152.108478
1,Carro Renault,4500.00,Euro,5.5465,24959.250000,2.00,54000.000,49918.500000
2,Notebook Dell,899.99,Dólar,5.1087,4597.778913,1.70,7649.915,7816.224152
3,IPhone,799.00,Dólar,5.1087,4081.851300,1.70,6791.500,6939.147210
4,Carro Fiat,3000.00,Euro,5.5465,16639.500000,1.90,34200.000,31615.050000
5,Celular Xiaomi,480.48,Dólar,5.1087,2454.628176,2.00,4804.800,4909.256352
6,Joia 20g,20.00,Ouro,312.2600,6245.200000,1.15,8050.000,7181.980000


### Agora vamos exportar a nova base de preços atualizada

In [4]:
#Passo6: Exportar a base calculada
tabela.to_excel("Produtos Novo.xlsx", index=False)

In [5]:
!pip install selenium